In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm 
import pprint as pp

In [2]:
TRAIN_DIR = 'train'#'X:/Kaggle_Data/dogs_vs_cats/train/train'
TEST_DIR = 'test'#'X:/Kaggle_Data/dogs_vs_cats/test/test'
IMG_SIZE = 64
LR = 1e-3
train_path = "/Users/jian/Desktop/CNN-TinyImagenet/tiny-imagenet-200/train"
label_path = "/Users/jian/Desktop/CNN-TinyImagenet/tiny-imagenet-200"

In [3]:
train_data =[]
label = []
counter = 0
for file in tqdm(os.listdir(train_path)):
    if counter == 10:
        break
    for pic in os.listdir(train_path+"/"+file+"/images"):
        img = cv2.imread(train_path+"/"+file+"/images/"+pic)
        train_data.append(np.array(img))
        label.append(file)
    counter += 1
train_data = np.array(train_data)
# np.save('tiny-imagenet-train.npy', train_data)

  5%|▌         | 10/200 [00:00<00:15, 12.30it/s]


In [4]:
label = np.array(label)

label = np.reshape(label, (len(label), 1))


In [5]:
lab_id = {}
lab = []
seen = []
labels = []
counter = 0
for l in tqdm(range(label.shape[0])):
    if counter == 10:
        break
    if label[l] in seen:
        continue
    else:
        seen.append(label[l])
        l_id = str(label[l])
        new_dict_entry = {l_id : counter}
        counter += 1 
        lab_id.update(new_dict_entry)
# print(lab_id) 
num_classes = counter
print(num_classes)

  0%|          | 0/5000 [00:00<?, ?it/s]

10


In [6]:
for l in tqdm(range(label.shape[0])):
    temp = np.zeros(num_classes)
    clss = label[l]
    index = lab_id.get(str(clss))
    temp[index] = 1
    labels.append(temp)

100%|██████████| 5000/5000 [00:00<00:00, 51757.76it/s]


In [7]:
labels = np.array(labels)
# print(labels.shape)

In [8]:
training_data = []
for l in tqdm(range(label.shape[0])):
    training_data.append([train_data[l], labels[l]])
    
shuffle(training_data)

np.save('training_data.npy', training_data)


100%|██████████| 5000/5000 [00:00<00:00, 702234.13it/s]


In [9]:
import data_augmentation as da
DA_FACTOR = 8
trn_data = np.load('training_data.npy')
print(trn_data.shape)
print(trn_data[0].shape)
# print(trn_data[0])

ag_trn_data = np.empty([DA_FACTOR,2])
# print(trn_data[0])
for img in trn_data:
    new_data = np.empty([DA_FACTOR,2])
    new_data = da.data_aug(img)
#     print('new_data',new_data.shape)
#     print('ag_train_data before concate',ag_trn_data.shape)
    ag_trn_data = np.concatenate([ag_trn_data, new_data])
print ('date augmented successfully', ag_trn_data.shape)
del_indices = range(DA_FACTOR)
# print('del indices',del_indices)
ag_trn_data = np.delete(ag_trn_data, del_indices, 0)
print('newx shape',ag_trn_data.shape)
##todo:check new_x[2] should equal to new_data(X[0])
trn_data = ag_trn_data
shuffle(trn_data)

(5000, 2)
(2,)
date augmented successfully (40008, 2)
newx shape (40000, 2)


In [10]:
import tflearn
from tflearn.layers.normalization import local_response_normalization as lrn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

convnet = conv_2d(convnet, 64, [11,11],  4, activation='relu')
convnet = lrn(convnet, depth_radius=5, bias=2.0,alpha = 1e-4,beta=0.75)
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = conv_2d(convnet, 192, [5,5], activation='relu')
convnet = lrn(convnet, depth_radius=5, bias=2.0,alpha = 1e-4,beta=0.75)
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = conv_2d(convnet, 384, [3,3], activation='relu')

convnet = conv_2d(convnet, 384, [3,3], activation='relu')

convnet = conv_2d(convnet, 256, [3,3], activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = fully_connected(convnet, 1024, activation='relu')

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.5)

convnet = fully_connected(convnet, num_classes, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log',max_checkpoints=1, tensorboard_verbose=0)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [11]:
# print(trn_data)
train = trn_data[:-500]
test = trn_data[-500:]

In [12]:
x = np.array([i[0] for i in train])
# print(x.shape)
y = [i[1] for i in train]
# print(y[0])

test_x = np.array([i[0] for i in test])
test_y = [i[1] for i in test]

In [13]:
MODEL_NAME = 'test_cnn-{}-{}.model'.format(LR, 'data_aug_8-step_500-4-testingtheory')
model.fit({'input': x}, {'targets': y}, n_epoch=40, validation_set=({'input': test_x}, {'targets': test_y})
          ,snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 5701  | total loss: 0.19498 | time: 41.675s
| Adam | epoch: 010 | loss: 0.19498 - acc: 0.9466 -- iter: 08896/39500


KeyboardInterrupt: 